# EJERCICIOS - SESIÓN 2: Inyección de Dependencias y Arquitectura Modular

**Objetivo:** Practicar los conceptos de inyección de dependencias, yield, servicios, APIRouter y sub-dependencias.

**Instrucciones:**
- Completa los TODOs en cada ejercicio
- Usa `Annotated` para definir tipos reutilizables
- Ejecuta las celdas de prueba para validar tu código
- Los ejercicios son independientes entre sí

## CONFIGURACIÓN DEL ENTORNO

In [ ]:
# Verificación de Python
import sys
print(f"Python: {sys.version}")
assert sys.version_info >= (3, 8), "Se requiere Python 3.8 o superior"

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 uvicorn[standard]==0.32.0 -q
print("Dependencias instaladas")

In [ ]:
# Importaciones globales
from fastapi import FastAPI, Depends, HTTPException, Header, APIRouter, status, Query
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field
from typing import Optional, Annotated, Generator, List
import time

print("Imports completados")

---

## EJERCICIO 1: Dependencia Simple que Lee Header

**Objetivo:** Crear una dependencia que extraiga y valide un header HTTP.

**Requisitos:**
- Crea una dependencia `obtener_idioma()` que lea el header `accept-language`
- Si el header no está presente, debe devolver "es" por defecto
- Define un tipo `IdiomaDep` usando `Annotated`
- Crea un endpoint `/saludo` que use esta dependencia y devuelva un saludo en el idioma correspondiente

In [ ]:
# TODO: Implementa la dependencia obtener_idioma
def obtener_idioma(accept_language: str = Header(default="es")):
    # TODO: Devuelve el idioma (puede ser simplemente el valor del header)
    pass

# TODO: Define el tipo reutilizable IdiomaDep
# IdiomaDep = ...

app = FastAPI()

# TODO: Implementa el endpoint /saludo
# Debe devolver {"mensaje": "Hola"} si idioma="es", {"mensaje": "Hello"} si idioma="en"
@app.get("/saludo")
def saludar():
    # TODO: Usa IdiomaDep como parámetro
    pass

# Pruebas
# client = TestClient(app)
# 
# # Prueba 1: Idioma por defecto (español)
# r1 = client.get("/saludo")
# assert r1.status_code == 200
# assert r1.json()["mensaje"] == "Hola", f"Error: {r1.json()}"
# print("✅ Prueba 1 pasada: Saludo en español")
# 
# # Prueba 2: Idioma inglés
# r2 = client.get("/saludo", headers={"accept-language": "en"})
# assert r2.json()["mensaje"] == "Hello"
# print("✅ Prueba 2 pasada: Saludo en inglés")

---

## EJERCICIO 2: Dependencia con Validación

**Objetivo:** Crear una dependencia que valide datos y lance excepciones HTTP.

**Requisitos:**
- Crea una dependencia `validar_api_key()` que lea el header `x-api-key`
- Si la API key no es "clave-secreta-123", lanza `HTTPException` con código 403
- Define un tipo `ApiKeyValidada` usando `Annotated`
- Crea un endpoint `/datos-privados` que use esta dependencia

In [ ]:
# TODO: Implementa la dependencia validar_api_key
def validar_api_key():
    # TODO: Lee el header x-api-key
    # TODO: Si no es "clave-secreta-123", lanza HTTPException(status_code=403, detail="API key inválida")
    # TODO: Devuelve la API key validada
    pass

# TODO: Define el tipo ApiKeyValidada
# ApiKeyValidada = ...

app = FastAPI()

# TODO: Implementa el endpoint /datos-privados
@app.get("/datos-privados")
def obtener_datos_privados():
    # TODO: Usa ApiKeyValidada como parámetro
    # TODO: Devuelve {"datos": "información confidencial", "api_key": api_key}
    pass

# Pruebas
# client = TestClient(app)
# 
# # Prueba 1: API key válida
# r1 = client.get("/datos-privados", headers={"x-api-key": "clave-secreta-123"})
# assert r1.status_code == 200
# assert "información confidencial" in r1.json()["datos"]
# print("✅ Prueba 1 pasada: API key válida")
# 
# # Prueba 2: API key inválida (debe dar 403)
# r2 = client.get("/datos-privados", headers={"x-api-key": "incorrecta"})
# assert r2.status_code == 403, f"Debería ser 403, pero fue {r2.status_code}"
# print("✅ Prueba 2 pasada: API key inválida rechazada")

---

## EJERCICIO 3: Clase Servicio Inyectable

**Objetivo:** Crear un servicio con lógica de negocio e inyectarlo en endpoints.

**Requisitos:**
- Crea una clase `CalculadoraService` con métodos `sumar(a, b)` y `multiplicar(a, b)`
- Crea una dependencia `obtener_calculadora()` que devuelva una instancia de `CalculadoraService`
- Define un tipo `CalculadoraDep`
- Crea dos endpoints: `/sumar?a=5&b=3` y `/multiplicar?a=4&b=2`

In [ ]:
# TODO: Implementa la clase CalculadoraService
class CalculadoraService:
    def sumar(self, a: int, b: int) -> int:
        # TODO: Retorna la suma
        pass
    
    def multiplicar(self, a: int, b: int) -> int:
        # TODO: Retorna el producto
        pass

# TODO: Implementa la dependencia
def obtener_calculadora() -> CalculadoraService:
    # TODO: Devuelve una instancia de CalculadoraService
    pass

# TODO: Define el tipo CalculadoraDep
# CalculadoraDep = ...

app = FastAPI()

# TODO: Implementa el endpoint /sumar
@app.get("/sumar")
def sumar_numeros():
    # TODO: Parámetros a, b y calculadora: CalculadoraDep
    # TODO: Devuelve {"resultado": calculadora.sumar(a, b)}
    pass

# TODO: Implementa el endpoint /multiplicar
@app.get("/multiplicar")
def multiplicar_numeros():
    # TODO: Similar a sumar
    pass

# Pruebas
# client = TestClient(app)
# 
# # Prueba sumar
# r1 = client.get("/sumar?a=5&b=3")
# assert r1.status_code == 200
# assert r1.json()["resultado"] == 8
# print("✅ Suma correcta: 5 + 3 = 8")
# 
# # Prueba multiplicar
# r2 = client.get("/multiplicar?a=4&b=2")
# assert r2.json()["resultado"] == 8
# print("✅ Multiplicación correcta: 4 * 2 = 8")

---

## EJERCICIO 4: Dependencia con Yield (Simular BD)

**Objetivo:** Crear una dependencia con yield para gestionar recursos.

**Requisitos:**
- Crea una clase `FakeCache` con métodos `connect()`, `disconnect()`, `get(key)`, `set(key, value)`
- Crea una dependencia `obtener_cache()` que use yield para conectar/desconectar
- Define un tipo `CacheDep`
- Crea endpoints `/cache/set?key=x&value=y` y `/cache/get?key=x`

In [ ]:
# TODO: Implementa la clase FakeCache
class FakeCache:
    def __init__(self):
        self.connected = False
        self.data = {}
    
    def connect(self):
        # TODO: Imprime "🔌 Conectando cache..." y marca connected = True
        pass
    
    def disconnect(self):
        # TODO: Imprime "🔌 Desconectando cache..." y marca connected = False
        pass
    
    def get(self, key: str):
        # TODO: Retorna self.data.get(key) o None
        pass
    
    def set(self, key: str, value: str):
        # TODO: Guarda en self.data[key] = value
        pass

# TODO: Implementa la dependencia con yield
def obtener_cache() -> Generator[FakeCache, None, None]:
    # TODO: Crea instancia, connect(), yield, finally disconnect()
    pass

# TODO: Define el tipo CacheDep
# CacheDep = ...

app = FastAPI()

# TODO: Implementa /cache/set
@app.post("/cache/set")
def guardar_en_cache():
    # TODO: Parámetros key, value, cache: CacheDep
    # TODO: cache.set(key, value) y devuelve {"mensaje": "guardado"}
    pass

# TODO: Implementa /cache/get
@app.get("/cache/get")
def obtener_de_cache():
    # TODO: Parámetro key, cache: CacheDep
    # TODO: Devuelve {"valor": cache.get(key)}
    pass

# Pruebas
# client = TestClient(app)
# 
# # Guardar valor en cache
# r1 = client.post("/cache/set?key=nombre&value=Ana")
# assert r1.status_code == 200
# assert r1.json()["mensaje"] == "guardado"
# print("✅ Valor guardado en cache")
# 
# # Recuperar valor (⚠️ NOTA: El cache se reinicia en cada request, ¡esperar None!)
# r2 = client.get("/cache/get?key=nombre")
# assert r2.status_code == 200
# print(f"⚠️ Valor recuperado: {r2.json()} (puede ser None por nueva instancia)")

---

## EJERCICIO 5: Sub-dependencia

**Objetivo:** Crear una cadena de dependencias donde una depende de otra.

**Requisitos:**
- Dependencia 1: `obtener_timestamp()` que devuelve la hora actual (usa `time.time()`)
- Dependencia 2: `obtener_logger(timestamp)` que depende de la anterior y devuelve un dict con el timestamp
- Define tipos `TimestampDep` y `LoggerDep`
- Endpoint `/log` que use `LoggerDep` y devuelva información del log

In [ ]:
# TODO: Implementa obtener_timestamp
def obtener_timestamp() -> float:
    # TODO: Devuelve time.time()
    pass

# TODO: Define TimestampDep
# TimestampDep = ...

# TODO: Implementa obtener_logger (depende de TimestampDep)
def obtener_logger():
    # TODO: Recibe timestamp: TimestampDep como parámetro
    # TODO: Devuelve {"timestamp": timestamp, "nivel": "INFO"}
    pass

# TODO: Define LoggerDep
# LoggerDep = ...

app = FastAPI()

# TODO: Implementa /log
@app.get("/log")
def obtener_log():
    # TODO: Usa logger: LoggerDep
    # TODO: Devuelve {"log": logger, "mensaje": "Registro creado"}
    pass

# Pruebas
# client = TestClient(app)
# 
# r = client.get("/log")
# assert r.status_code == 200
# assert "log" in r.json()
# assert "timestamp" in r.json()["log"]
# assert r.json()["log"]["nivel"] == "INFO"
# print("✅ Log generado correctamente:", r.json())

---

## EJERCICIO 6: APIRouter Básico

**Objetivo:** Crear un router básico y registrarlo en la aplicación.

**Requisitos:**
- Crea un `APIRouter` sin prefijo ni tags
- Agrega dos endpoints al router: `/items` (GET) y `/items/{item_id}` (GET)
- Registra el router en la aplicación FastAPI

In [ ]:
# TODO: Crea el router
# router = ...

# TODO: Implementa GET /items
# @router.get("/items")
# def listar_items():
#     # Devuelve {"items": ["item1", "item2", "item3"]}
#     pass

# TODO: Implementa GET /items/{item_id}
# @router.get("/items/{item_id}")
# def obtener_item():
#     # Devuelve {"item_id": item_id, "nombre": f"Item {item_id}"}
#     pass

app = FastAPI()

# TODO: Registra el router
# app.include_router(...)

# Pruebas
# client = TestClient(app)
# 
# r1 = client.get("/items")
# assert r1.status_code == 200
# assert "items" in r1.json()
# print("✅ Listar items:", r1.json())
# 
# r2 = client.get("/items/5")
# assert r2.json()["item_id"] == 5
# print("✅ Obtener item específico:", r2.json())

---

## EJERCICIO 7: Router con Prefijo y Tags

**Objetivo:** Crear un router con configuración de prefijo y tags.

**Requisitos:**
- Crea un router con prefijo `/api/v1/productos` y tag "Productos"
- Agrega endpoints: `/` (GET lista), `/` (POST crear), `/{id}` (DELETE eliminar)
- Registra el router en la aplicación

In [ ]:
# TODO: Crea el router con prefix y tags
# router_productos = APIRouter(
#     prefix=...,
#     tags=[...]
# )

# TODO: Implementa GET / (lista)
# @router_productos.get("/")
# def listar_productos():
#     pass

# TODO: Implementa POST / (crear)
# @router_productos.post("/")
# def crear_producto():
#     # Parámetro nombre: str
#     # Devuelve {"id": 1, "nombre": nombre, "mensaje": "creado"}
#     pass

# TODO: Implementa DELETE /{id}
# @router_productos.delete("/{producto_id}")
# def eliminar_producto():
#     # Devuelve {"mensaje": f"Producto {producto_id} eliminado"}
#     pass

app = FastAPI()

# TODO: Registra el router
# app.include_router(...)

# Pruebas
# client = TestClient(app)
# 
# r1 = client.get("/api/v1/productos")
# assert r1.status_code == 200
# print("✅ Listar productos:", r1.json())
# 
# r2 = client.post("/api/v1/productos?nombre=Laptop")
# assert r2.json()["nombre"] == "Laptop"
# print("✅ Producto creado:", r2.json())
# 
# r3 = client.delete("/api/v1/productos/1")
# assert "eliminado" in r3.json()["mensaje"]
# print("✅ Producto eliminado:", r3.json())

---

## EJERCICIO 8: Dependencias Globales en Router

**Objetivo:** Aplicar una dependencia a todos los endpoints de un router.

**Requisitos:**
- Crea una dependencia `verificar_rol_admin()` que lea header `x-role` y valide que sea "admin"
- Crea un router `/admin` con esta dependencia aplicada globalmente
- Agrega dos endpoints: `/usuarios` y `/configuracion`

In [ ]:
# TODO: Implementa verificar_rol_admin
def verificar_rol_admin():
    # TODO: Lee x-role del header
    # TODO: Si no es "admin", lanza HTTPException 403
    # TODO: Devuelve True
    pass

# TODO: Crea el router con dependencia global
# router_admin = APIRouter(
#     prefix="/admin",
#     tags=["Admin"],
#     dependencies=[Depends(verificar_rol_admin)]  # <-- Dependencia global
# )

# TODO: Implementa GET /usuarios
# @router_admin.get("/usuarios")
# def admin_usuarios():
#     # Devuelve {"usuarios": ["todos los usuarios del sistema"]}
#     pass

# TODO: Implementa GET /configuracion
# @router_admin.get("/configuracion")
# def admin_configuracion():
#     # Devuelve {"config": {"modo": "producción", "debug": False}}
#     pass

app = FastAPI()

# TODO: Registra el router
# app.include_router(...)

# Pruebas
# client = TestClient(app)
# 
# # Con rol admin (debe funcionar)
# r1 = client.get("/admin/usuarios", headers={"x-role": "admin"})
# assert r1.status_code == 200
# print("✅ Acceso admin a /usuarios:", r1.json())
# 
# # Con rol user (debe dar 403)
# r2 = client.get("/admin/usuarios", headers={"x-role": "user"})
# assert r2.status_code == 403, f"Debería ser 403, pero fue {r2.status_code}"
# print("✅ Acceso denegado correctamente")
# 
# # Endpoint configuración con admin
# r3 = client.get("/admin/configuracion", headers={"x-role": "admin"})
# assert "config" in r3.json()
# print("✅ Acceso admin a /configuracion:", r3.json())

---

## EJERCICIO 9: Servicio Completo con Múltiples Métodos

**Objetivo:** Crear un servicio complejo con múltiples métodos de lógica de negocio.

**Requisitos:**
- Crea `TareaService` con almacenamiento en memoria (lista de dict)
- Métodos: `crear(titulo)`, `listar()`, `completar(id)`, `eliminar(id)`
- Cada tarea: `{"id": int, "titulo": str, "completada": bool}`
- Crea endpoints correspondientes a cada método

In [ ]:
# ⚠️ IMPORTANTE: Simulación de persistencia
# En FastAPI, Depends(Clase) crea una NUEVA INSTANCIA por cada request.
# Si guardamos datos en self.tareas = [], se pierden al terminar el request.
# Solución: Variable global que simula una BD externa (en producción sería SQL/MongoDB)

TAREAS_DB = []  # Simula base de datos externa
NEXT_ID = {"valor": 1}  # Contador global (dict para mutabilidad)

# TODO: Implementa TareaService
class TareaService:
    def __init__(self):
        # IMPORTANTE: Usamos la BD global, no creamos una nueva lista
        self.tareas = TAREAS_DB  # Referencia a la BD global
        self.next_id = NEXT_ID   # Referencia al contador global
    
    def crear(self, titulo: str) -> dict:
        # TODO: Crea una tarea {"id": self.next_id["valor"], "titulo": titulo, "completada": False}
        # TODO: Agrégala a self.tareas, incrementa self.next_id["valor"], devuelve la tarea
        pass
    
    def listar(self) -> List[dict]:
        # TODO: Devuelve self.tareas
        pass
    
    def completar(self, tarea_id: int) -> dict:
        # TODO: Busca la tarea por id en self.tareas, marca completada=True
        # TODO: Si no existe, lanza HTTPException(status_code=404, detail="Tarea no encontrada")
        # TODO: Devuelve la tarea modificada
        pass
    
    def eliminar(self, tarea_id: int) -> dict:
        # TODO: Busca y elimina la tarea de self.tareas
        # TODO: Si no existe, lanza HTTPException 404
        # TODO: Devuelve {"mensaje": "Tarea eliminada"}
        pass

# TODO: Dependencia y tipo
def obtener_tarea_service() -> TareaService:
    return TareaService()

# TareaServiceDep = ...

app = FastAPI()

# TODO: Implementa POST /tareas (parámetro titulo: str, service: TareaServiceDep)
# TODO: Implementa GET /tareas (service: TareaServiceDep)
# TODO: Implementa PUT /tareas/{tarea_id}/completar (tarea_id: int, service: TareaServiceDep)
# TODO: Implementa DELETE /tareas/{tarea_id} (tarea_id: int, service: TareaServiceDep)

# Pruebas
# TAREAS_DB.clear()  # Limpia BD antes de probar
# NEXT_ID["valor"] = 1
# client = TestClient(app)
# r1 = client.post("/tareas?titulo=Comprar pan")
# assert r1.status_code == 200, f"Error POST: {r1.status_code}"
# print("✅ Tarea creada:", r1.json())
# 
# r2 = client.get("/tareas")
# assert len(r2.json()) == 1, "La tarea no se guardó"
# print("✅ Tareas listadas:", r2.json())
# 
# r3 = client.put("/tareas/1/completar")
# assert r3.json()["completada"] == True, "No se marcó como completada"
# print("✅ Tarea completada:", r3.json())
# 
# r4 = client.delete("/tareas/1")
# assert r4.status_code == 200, "Error al eliminar"
# print("✅ Tarea eliminada:", r4.json())

---

## EJERCICIO 10: Arquitectura Completa

**Objetivo:** Combinar Router + Servicio + Dependencia con yield.

**Requisitos:**
- Crea `FakeDatabase` con `connect()`, `disconnect()`, `query(sql)`
- Crea dependencia `obtener_db()` con yield
- Crea `ProductoService(db)` con métodos `listar()`, `crear(nombre, precio)`
- Crea `APIRouter` `/api/productos` con endpoints que usen el servicio
- El servicio debe recibir la BD como dependencia

In [ ]:
# ⚠️ IMPORTANTE: Mismo concepto de persistencia que Ejercicio 9
# La BD se guarda en una variable global para persistir entre requests

DB_PRODUCTOS = [
    {"id": 1, "nombre": "Laptop", "precio": 1000},
    {"id": 2, "nombre": "Mouse", "precio": 25}
]  # Simula BD externa

# TODO: Implementa FakeDatabase
class FakeDatabase:
    def __init__(self):
        self.connected = False
        self.productos = DB_PRODUCTOS  # Referencia a la BD global
    
    def connect(self):
        # TODO: Imprime "📦 Conectando BD..." y marca self.connected=True
        pass
    
    def disconnect(self):
        # TODO: Imprime "📦 Desconectando BD..." y marca self.connected=False
        pass
    
    def query(self, sql: str):
        # TODO: Simplemente devuelve self.productos (ignoramos SQL por simplicidad)
        pass

# TODO: Dependencia con yield
def obtener_db() -> Generator[FakeDatabase, None, None]:
    # TODO: Crea instancia FakeDatabase, llama connect(), yield instancia, finally disconnect()
    pass

# DatabaseDep = Annotated[FakeDatabase, Depends(obtener_db)]

# TODO: Implementa ProductoService
class ProductoService:
    def __init__(self, db: FakeDatabase):
        self.db = db
    
    def listar(self) -> List[dict]:
        # TODO: return self.db.query("SELECT * FROM productos")
        pass
    
    def crear(self, nombre: str, precio: float) -> dict:
        # TODO: Calcula nuevo_id = max(p["id"] for p in self.db.productos) + 1 si hay productos, sino 1
        # TODO: nuevo_producto = {"id": nuevo_id, "nombre": nombre, "precio": precio}
        # TODO: Agrégalo a self.db.productos y devuélvelo
        pass

# TODO: Dependencia del servicio (inyecta DatabaseDep)
def obtener_producto_service(db: "DatabaseDep"):
    # TODO: return ProductoService(db)
    pass

# ProductoServiceDep = Annotated[ProductoService, Depends(obtener_producto_service)]

# TODO: Crea el router
# router_productos = APIRouter(prefix="/api/productos", tags=["Productos"])

# TODO: GET / (listar)
# @router_productos.get("/")
# def listar_productos(service: "ProductoServiceDep"):
#     # return service.listar()
#     pass

# TODO: POST / (crear)
# @router_productos.post("/")
# def crear_producto(nombre: str, precio: float, service: "ProductoServiceDep"):
#     # return service.crear(nombre, precio)
#     pass

app = FastAPI()

# TODO: Registra el router
# app.include_router(router_productos)

# Pruebas
# client = TestClient(app)
# 
# # Listar productos iniciales
# r1 = client.get("/api/productos")
# assert r1.status_code == 200
# assert len(r1.json()) == 2, "Deberían existir 2 productos iniciales"
# print("✅ Productos iniciales:", r1.json())
# 
# # Crear nuevo producto
# r2 = client.post("/api/productos?nombre=Teclado&precio=75")
# assert r2.status_code == 200
# assert r2.json()["nombre"] == "Teclado"
# print("✅ Producto creado:", r2.json())
# 
# # Verificar que se guardó (debe haber 3 ahora)
# r3 = client.get("/api/productos")
# assert len(r3.json()) == 3, "Ahora deberían ser 3 productos"
# print("✅ Producto persistió correctamente:", r3.json())